In [1]:
import os

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen

In [2]:
context = InteractiveContext()

In [68]:
base_dir = os.getcwd()
data_dir = "../data/"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 33
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-04T14_41_47.870040-fgcctkgw/CsvExampleGen/examples/33/train/, split: train, id: 27)
            Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-04T14_41_47.870040-fgcctkgw/CsvExampleGen/examples/33/eval/, split: eval, id: 28)]
        ))

In [69]:
""" Loading from GCP bucket
"""

# examples = external_input("gs://example_compliance_data/")
# example_gen = CsvExampleGen(input=examples)

# context.run(example_gen)

' Loading from GCP bucket\n'

In [70]:
""" Ingesting data from GCP BigQuery
"""

# !export GOOGLE_APPLICATION_CREDENTIALS=/Users/user/credentials.json
# or
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/user/credentials.json"

# from tfx.components.example_gen.big_query_example_gen.component import BigQueryExampleGen  

# query = """SELECT * FROM `complaint-pipeline.consumer_complaints_22Jan.complaints`"""
# example_gen = BigQueryExampleGen(query=query)

# context.run(example_gen)

' Ingesting data from GCP BigQuery\n'

In [71]:
""" Three-way data split
"""

# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# base_dir = os.getcwd()
# data_dir = "../data/"
# output = example_gen_pb2.Output( 
#              split_config=example_gen_pb2.SplitConfig(splits=[
#                  example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
#                  example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
#                  example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
#              ]))

# examples = external_input(os.path.join(base_dir, data_dir)) 
# example_gen = CsvExampleGen(input=examples, output_config=output)

# context.run(example_gen)

' Three-way data split\n'

In [72]:
for artifact in example_gen.outputs['examples'].get():
    print(artifact)

Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-04T14_41_47.870040-fgcctkgw/CsvExampleGen/examples/33/train/, split: train, id: 27)
Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-04T14_41_47.870040-fgcctkgw/CsvExampleGen/examples/33/eval/, split: eval, id: 28)


In [73]:
""" Input example
"""

# import os
# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
# context = InteractiveContext()

# base_dir = os.getcwd()
# data_dir = "../data/"

# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(name='train', pattern='train/*'),
#                 example_gen_pb2.Input.Split(name='eval', pattern='eval/*'),
#                 example_gen_pb2.Input.Split(name='test', pattern='test/*')
#             ])

# examples = external_input(os.path.join(base_dir, data_dir)) 
# example_gen = CsvExampleGen(input=examples, input_config=input)
# context.run(example_gen)

' Input example\n'

In [74]:
""" Span Loading Example
"""

# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# base_dir = os.getcwd()
# data_dir = "../data3/"

# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(pattern='export-{SPAN}/*')
#             ])

# examples = external_input(os.path.join(base_dir, data_dir)) 

# example_gen = CsvExampleGen(input=examples, input_config=input)
# context.run(example_gen)

' Span Loading Example\n'

In [75]:
from tfx.components.statistics_gen.component import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

In [76]:
from tfx.components.schema_gen.component import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_disputed',STRING,optional,single,'consumer_disputed'
'date_received',STRING,required,,'date_received'
'date_sent_to_company',STRING,required,,'date_sent_to_company'
'issue',STRING,optional,single,'issue'
'product',STRING,required,,'product'
'state',STRING,optional,single,'state'
'sub_issue',STRING,optional,single,'sub_issue'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'In progress', 'Untimely response'"
'consumer_disputed',"'No', 'Yes'"
'date_received',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'date_sent_to_company',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender repossessed or 

In [77]:
from tfx.components.example_validator.component import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'issue',Unexpected string values,"Examples contain values missing from the schema: Cash advance (<1%), Lender damaged or destroyed vehicle (<1%), Unexpected/Other fees (<1%)."


In [86]:
# %%skip_for_export
%%writefile transform.py

import tensorflow as tf 
import tensorflow_transform as tft


_ONE_HOT_FEATURE_KEYS = [
    "product", "company_response", "timely_response"
]

_ONE_HOT_FEATURE_DIMS = [11, 6, 2]

_OOV_SIZE = 10

_CATEGORICAL_FEATURE_KEYS = [
    "sub_product", "state", "zip_code", # "consumer_disputed"
]

_MAX_CATEGORICAL_FEATURE_VALUES = [41, 59, 90000, 2] 

_TEXT_FEATURE_KEYS = [
    "issue", "sub_issue", "company"
]

_LABEL_KEY = 'consumer_disputed'


def _transformed_name(key):
    return key + '_xf'


def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]


def _fill_in_missing(x, to_string=False, unk=""):
    """Replace missing values in a SparseTensor.

    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.

    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
        in the second dimension.

    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    if to_string:
        x = tf.cast(x, tf.string)
    default_value = '' if x.dtype == tf.string or to_string else 0

    # if type(x) == tf.SparseTensor:
    #     x = tf.sparse.to_dense(
    #         tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), default_value)
    # return tf.squeeze(x, axis=1)

    if type(x) == tf.Tensor:
        return tf.squeeze(x, axis=1)
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value), axis=1)

def preprocessing_text(text):
    # let's lower all input strings
    text = tf.strings.lower(text)
    
    # Before applying the word piece tokenization, let's remove unnecessary 
    # tokens. This regex_replace was suggested by Aurélien Geron in his 
    # TFX workshop at TensorFlow World 2019
    # https://github.com/tensorflow/workshops/blob/master/tfx_labs/Lab_10_Neural_Structured_Learning.ipynb
    text = tf.strings.regex_replace(text, r" '| '|^'|'$", " ")
    text = tf.strings.regex_replace(text, "[^a-z' ]", " ")
    return text


def convert_num_to_one_hot(label_tensor, num_labels=2):
    """
    Convert a label (0 or 1) into a one-hot vector
    Args:
        int: label_tensor (0 or 1)
    Returns
        label tensor
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])


def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.

    Args:
      inputs: map from feature keys to raw not-yet-transformed features.

    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}

    for i, key in enumerate(_CATEGORICAL_FEATURE_KEYS):
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key], to_string=True),
            top_k=_MAX_CATEGORICAL_FEATURE_VALUES[i],
            num_oov_buckets=_OOV_SIZE)

    for i, key in enumerate(_ONE_HOT_FEATURE_KEYS):
        int_value = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key], to_string=True),
            top_k=_ONE_HOT_FEATURE_DIMS[i] + 1)
        outputs[_transformed_name(key)] = convert_num_to_one_hot(
            int_value,
            num_labels=_ONE_HOT_FEATURE_DIMS[i] + 1
        )

    for key in _TEXT_FEATURE_KEYS:
        outputs[_transformed_name(key)] = preprocessing_text(
            _fill_in_missing(inputs[key], to_string=True)
        )

    return outputs



Overwriting transform.py


In [87]:
from tfx.components.transform.component import Transform
# from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-04T14_41_47.870040-fgcctkgw/Transform/transform_output/41/.temp_path/tftransform_tmp/9cf490f1ef8c46d3adf837a07bd5f434/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-04T14_41_47.870040-fgcctkgw/Transform/transform_output/41/.temp_path/tftransform_tmp/26255990c16a48ee893575ecb9dd6bb6/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
ERROR:tensorflow:Cast int64 to string is not supported
	 [[node transform/Cast_2 (defined at usr/local/lib/python3.7/site-packages/tensorflow_transform/saved/saved_transform_io.py:240) ]]

Original stack trace for 'transform/Cast_2':
  File "usr/local/Cellar/python/3.7.5

ValueError: bad inputs: [[array([b'In progress'], dtype=object)], [array([b'Debt collection'], dtype=object)], SparseTensorValue(indices=[(0, 0)], values=[b'TX'], dense_shape=(1, 1)), SparseTensorValue(indices=array([], shape=(0, 2), dtype=int64), values=[], dense_shape=(1, 0)), [array([b'Yes'], dtype=object)], [array([76119])]] [while running 'Analyze/ApplySavedModel[Phase0]/ApplySavedModel']